$$
% transferring macros:
\let\eps\varepsilon
\let\wt\widetilde
\let\tn\mathrm
\let\vphi\varphi
\let\pa\partial
\let\para\parallel
\let\wh\widehat
\let\sl\shoveleft
% defining new macros:
\def\be{\begin{equation} \mskip 1cm}
\def\ee{\end{equation}}
\def\ba{\begin{align} \mskip 1cm}
\def\ea{\end{align}}
\def\RR{{\bf R}}
\def\grad{\tn{grad}\,}
\def\curl{\tn{curl}\,}
\def\div{\tn{div}\,}
\def\Im{\tn{Im}}
% defining new commands:
\newcommand{\abs}[1]{\lvert#1\rvert}
\newcommand{\bold}[1]{{\bf #1}}
\newcommand{\pder}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\dt}[1]{\frac{\mathrm d #1}{\mathrm dt}}
$$

# Basics of Lagrange finite elements 

In [2]:
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.integrate import fixed_quad
import fembase as fem

ImportError: No module named 'sympy'

## Shape functions 

In [ ]:
degree = 2
# polynomial degree
shapefun = fem.LagrangeShape(np.linspace(-1, 1, degree + 1))
# check the docstring to see what this does

In [ ]:
fem.LagrangeShape?
print(shapefun.__doc__)

In [ ]:
np.polyder?

In [ ]:
print('kind =', shapefun.kind)
print('\nd =', shapefun.d)
print('\ns =', shapefun.s)
print('\neta =', shapefun.eta)
print('\nDeta =', shapefun.Deta)
print('\nmass =', shapefun.mass)
print('\nstiff =', shapefun.stiff)

In [ ]:
#fig = plt.figure(figsize=(8, 10))
fig, axes = plt.subplots(2, 1, figsize=(8, 10))
#axes = fig.subplots(2, 1)
(ax1, ax2) = axes 

svec = np.linspace(-1,1,100)

for i in range(shapefun.d + 1):
    ax1.plot(svec, shapefun.eta[i](svec))
    ax2.plot(svec, shapefun.Deta[i](svec))
    
ax1.grid()
ax1.set(title='Lagrange shape functions of degree ' + str(shapefun.d))
ax2.grid()
ax2.set(title='Corresponding derivatives')

plt.show()

In [ ]:
fig,axes = plt.subplots(shapefun.d+1,1, figsize=(8, shapefun.d*6 + 2 ))
#axes = fig.subplots(shapefun.d + 1, 1) 

for i in range(shapefun.d + 1):
    
    if i == shapefun.d:
        axes[0].plot(svec - shapefun.s[i], shapefun.eta[i](svec), color='C' + str(i))
    else:
        axes[i].plot(svec - shapefun.s[i], shapefun.eta[i](svec), color='C' + str(i))
        axes[i].grid()
        axes[i].set(title='Basis function eta_' + str(i))

fig.delaxes(axes[shapefun.d])
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(12, 5))

for i in range(shapefun.d + 1):
    
    if i == 0:
        ax.plot(svec + 2, shapefun.eta[i](svec), color='C' + str(i))
        ax.plot(svec + 4, shapefun.eta[i](svec), color='C' + str(i))
    elif i == shapefun.d:
        ax.plot(svec, shapefun.eta[i](svec), color='C' + str(i))
        ax.plot(svec + 2, shapefun.eta[i](svec), color='C' + str(i))
    else:
        ax.plot(svec, shapefun.eta[i](svec), color='C' + str(i))
        ax.plot(svec + 2, shapefun.eta[i](svec), color='C' + str(i))
        ax.plot(svec + 4, shapefun.eta[i](svec), color='C' + str(i))
    
    
ax.grid()
ax.set(title='Lagrange basis functions of degree ' + str(shapefun.d))

plt.show()

## Global mass and stiffness matrices



In [ ]:
el_b = np.linspace(-np.pi, np.pi, 2**4 + 1)
# element boundaries (defining the simulation domain including the domain boundaries)
# can be any 1D array with strictly increasing entries
print('len(el_b) =', len(el_b))
print()
for i in range(len(el_b)):
    print('el_b({0}) = {1:3.2f}'.format(i, el_b[i]))

Nel = len(el_b) - 1
print('\nNel =', Nel)
# number of elements
deg = shapefun.d
# polynomial degree
NN = Nel*(deg + 1) - (Nel - 1)
N0 = NN - 2 
N1 = NN - 1
# number of degrees of freedom (NN including the boundary)
print('\nNN =', NN)

# mesh points

We want to test our implementation of the mass and stiffness matrices. In order to verify the mass matrix we test the $L^2$-projection. Let $E\in H^1_0((a,b))$ and denote by $\Lambda^2_0: H^1_0 \to V_0$ the $L^2$-projection on the finite dimensional subspace spanned by the basis functions $(\varphi_j^0)_{1 \leq j \leq N_0}$. The $L^2$-projection is defined by

$$
\be \label{L2proj}
 ( \Lambda^2_0 E - E,\, \varphi_j^0 )_{L^2} = 0 \qquad \forall\,\varphi_j^0\,.
\ee
$$

At the same time, we have

$$
\be
 (\Lambda^2_0 E)(x) = \sum_{i=1}^{N_0} e_i\,\varphi_i^0(x)\,,
\ee
$$

where $e = (e_i)_{1\leq i \leq N_0} \in \mathbb R^{N_0}$ holds the coefficients of the projected $E$ in the given basis. Moreover, we denote the mass matrix by $M^0 \in \mathbb R^{N_0 \times N_0}$ and define the vector $\overline E = (\overline E_j)_{1 \leq j \leq N_0} \in \mathbb R^{N_0}$ with $\overline E_j := (E,\,\varphi_j^0)_{L^2}$. With this, from \eqref{L2proj} we obtain

$$
\ba
 &( \Lambda^2_0 E,\, \varphi_j^0 )_{L^2} = ( E,\, \varphi_j^0 )_{L^2} \qquad \forall\,\varphi_j^0
 \\[1mm]
 &\Leftrightarrow\quad \sum_{i=1}^{N_0} e_i \int_a^b \varphi_i^0 \varphi_j^0\,\tn d x = \overline E_j \qquad \forall\,j   \nonumber
 \\[0mm]
 &\Leftrightarrow\quad \sum_{i=1}^{N_0} e_i \, M^0_{i,j} = \overline E_j \qquad \forall\,j  \nonumber
 \\[1mm]
 &\Leftrightarrow\quad (M^0)^\top \cdot e = \overline E   \nonumber
 \\[4mm]
 &\Leftrightarrow\quad e = (M^0)^{-1} \cdot \overline E \,.  \label{L2formula}
\ea
$$

To get to the last line we used that the mass matrix is symmetric. We shall compute the projection of a given function $E$ from formula \eqref{L2formula} and then compare $||\Lambda^2_0 E - E||_{L^2}$ for different degrees of polynomials and mesh resolution. For this we included the above assembly procedures as the function <i>fembase.lag_assemb</i> in the module _fembase.py_.

In [ ]:
# The function we project:
Efun = lambda x: np.sin(x)*np.exp(x/np.pi)
xvec = np.linspace(-np.pi, np.pi, 500)
fig,ax = plt.subplots(figsize=(12, 5))
ax.plot(xvec, Efun(xvec))
ax.grid()

In [ ]:
degree = 1
shapefun = fem.LagrangeShape(np.linspace(-1, 1, degree + 1))
# the object holding the Lagrange shape functions

# loop over different number of elements (mesh resolution):
cases = 5
L2error = np.zeros(cases)
Nel_vec = np.zeros(cases)
for i in range(3, 3 + cases):
    
    el_b = np.linspace(-np.pi, np.pi, 2**i + 1)
    
    Nel, Nbase, Ebar = fem.lag_L2prod(Efun, shapefun.eta, el_b)
    
    Nel, Nbase, mass, stiff = fem.lag_assemb(el_b, shapefun.mass, shapefun.stiff)
    
    evec = np.linalg.solve(mass, Ebar)
    testvec = np.zeros(Nbase) 
    
    fig, ax1 = plt.subplots(figsize=(12, 5))
    fig, ax2 = plt.subplots(figsize=(12, 5))

    
    for j in range(Nbase):   
        testvec[:] = 0 
        testvec[j] = 1
        # coeficients of the projection

        Nel, Nbase, basefun = fem.lag_fun(testvec, shapefun.eta, el_b)
        
        ax1.plot(xvec, basefun(xvec))
        ax1.grid()
        
    Nel, Nbase, Eproj = fem.lag_fun(evec, shapefun.eta, el_b)

    ax2.plot(xvec, Eproj(xvec))
    ax2.grid()

    ax2.plot(xvec, Efun(xvec))
    ax2.grid()
        
    # L2-error:
    Nel_vec[i - 3] = Nel
    L2error[i - 3] = 0
    integrand = lambda x: abs(Eproj(x) - Efun(x))**2
    for k in range(Nel):
        L2error[i - 3] += fixed_quad(integrand, el_b[k], el_b[k + 1])[0]

    L2error[i - 3] = np.sqrt(L2error[i - 3 ]) 
    print('L2-error =', L2error[i - 3], 'degree =', degree, 'Nel =', Nel)
    

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.loglog(Nel_vec, L2error, label='L2-error')
ax.loglog(Nel_vec, Nel_vec**(-(degree + 1)), label='x^' + str(-degree-1))
ax.set(title='Convergence rate of the L2-projection for degree-' + str(degree) 
       + ' elements', xlabel='Nel', ylabel='L2-error')
ax.legend()

As a next step we want to check the stiffness matrix by solving the 1D Poisson equation

$$
\be \label{Poisson}
 -\frac{\partial^2u}{\partial x^2}=f
\ee
$$

with homogeneous Dirichlet b.c. $u(0)=u(L)=0$. After partial integration the equivalent weak formulation reads: Find $u\in H_0^1$ such that

$$
\be \label{Poisson2}
 \int_a^b\frac{\partial u}{\partial x}\frac{\pa \varphi}{\pa x}\text{d}x=\int_a^b f\varphi\text{d}x, \qquad\forall\varphi\in H_0^1
\ee
$$

The discrete problemm is given by: Find $u_h\in V_0^1$ such that

$$
\ba 
 &\int_a^b\frac{\partial u_h}{\partial x}\frac{\pa \varphi_j}{\pa x}\text{d}x=\int_a^b f\varphi_j\text{d}x, \qquad\forall\varphi_j\in V_0^1.
 \\[1mm]
 &\Leftrightarrow\quad \sum_{i=1}^{N_0}u_i\int_a^b(\pa_x\varphi_i^0)(\pa_x\varphi_j^0)\text{d}x=\int_a^bf\varphi_j^0\text{d}x,\qquad\forall j \nonumber
 \\[1mm]
 &\Leftrightarrow\quad \sum_{i=1}^{N_0} u_i \, S^0_{i,j} = \overline f_j \qquad \forall\,j  \nonumber
 \\[1mm]
 &\Leftrightarrow\quad (S^0)^\top \cdot u = \overline f   \nonumber
 \\[4mm]
 &\Leftrightarrow\quad u = (S^0)^{-1} \cdot \overline f \,. \label{Poisson4} 
\ea
$$

To get to the last line we used that the stiffness matrix is symmetric. We shall compute the solution for a given function $f$ from formula \eqref{Poisson4} and then compare $||\Lambda^2_0 E - E||_{L^2}$ for different degrees of polynomials and mesh resolution. For this we included the above assembly procedures as the function <i>fembase.lag_assemb</i> in the module _fembase.py_.

In [ ]:
# The solution we want to get:
x = sp.symbols('x')
u = sp.sin(x)*sp.exp(x/np.pi)
f = -sp.diff(u,x,x)
f = sp.lambdify(x,f)
u = sp.lambdify(x,u)
xvec = np.linspace(-np.pi, np.pi, 500)
fig,ax = plt.subplots(figsize=(12, 5))
ax.plot(xvec, u(xvec), label = 'the solution u(x)')
ax.plot(xvec, f(xvec), label = 'the right-hand side f(x)')
ax.grid()
ax.legend()

In [ ]:
degree = 1
shapefun = fem.LagrangeShape(np.linspace(-1, 1, degree + 1))
# the object holding the Lagrange shape functions

# loop over different number of elements (mesh resolution):
cases = 5
L2error = np.zeros(cases)
Nel_vec = np.zeros(cases)
for i in range(3, 3 + cases):
    
    el_b = np.linspace(-np.pi, np.pi, 2**i + 1)
    
    Nel, Nbase, fbar = fem.lag_L2prod(f, shapefun.eta, el_b)
    
    Nel, Nbase, mass, stiff = fem.lag_assemb(el_b, shapefun.mass, shapefun.stiff)
    
    evec = np.linalg.solve(stiff, fbar)
    testvec = np.zeros(Nbase) 
    
    fig, ax1 = plt.subplots(figsize=(12, 5))
    fig, ax2 = plt.subplots(figsize=(12, 5))

    
    for j in range(Nbase):   
        testvec[:] = 0 
        testvec[j] = 1
        # coeficients of the projection

        Nel, Nbase, basefun = fem.lag_fun(testvec, shapefun.eta, el_b)
        
        ax1.plot(xvec, basefun(xvec))
        ax1.grid()
        
    Nel, Nbase, uh = fem.lag_fun(evec, shapefun.eta, el_b)

    ax2.plot(xvec, uh(xvec))
    ax2.grid()

    ax2.plot(xvec, u(xvec))
    ax2.grid()
        
    # L2-error:
    Nel_vec[i - 3] = Nel
    L2error[i - 3] = 0
    integrand = lambda x: abs(uh(x) - u(x))**2
    for k in range(Nel):
        L2error[i - 3] += fixed_quad(integrand, el_b[k], el_b[k + 1])[0]

    L2error[i - 3] = np.sqrt(L2error[i - 3 ]) 
    print('L2-error =', L2error[i - 3], 'degree =', degree, 'Nel =', Nel)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.loglog(Nel_vec, L2error, label='L2-error')
ax.loglog(Nel_vec, Nel_vec**(-(degree + 1)), label='x^' + str(-degree-1))
ax.set(title='Convergence rate of the Poisson solution for degree-' + str(degree) 
       + ' elements', xlabel='Nel', ylabel='L2-error')
ax.legend()